In [108]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
import torch
from transformers import set_seed
import warnings
warnings.filterwarnings("ignore")

try:
    print(f"PyTorch version: {torch.__version__}")
    print(f"PyTorch location: {torch.__file__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
except ImportError as e:
    print(f"Error importing torch: {e}")

PyTorch version: 2.6.0
PyTorch location: /Users/vlzqz/Desktop/escuela/spring 2025/440 social networks/topicQuestions/.venv/lib/python3.13/site-packages/torch/__init__.py
CUDA available: False


In [109]:
prompt = "It was a dark and stormy"
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2-0.5B")
input_ids = tokenizer(prompt).input_ids
input_ids

[2132, 572, 264, 6319, 323, 13458, 88]

In [110]:
for t in input_ids:
    print(t, "\t:", tokenizer.decode(t))

2132 	: It
572 	:  was
264 	:  a
6319 	:  dark
323 	:  and
13458 	:  storm
88 	: y


# Notes:


*  The SmolLM tokenizer does not split any of the words in this particular sentence.
*   Each model is usually paired with its own tokenizer, so always use the proper tokenizer when using a model.




In [111]:
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2-0.5B")

In [112]:
prompt

'It was a dark and stormy'

In [113]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

outputs = model(input_ids)
outputs.logits.shape  # An output for each input token

torch.Size([1, 7, 151936])

In [114]:
input_ids

tensor([[ 2132,   572,   264,  6319,   323, 13458,    88]])

In [115]:
outputs

CausalLMOutputWithPast(loss=None, logits=tensor([[[ 9.0280,  6.4770,  3.8583,  ..., -4.3870, -4.3866, -4.3871],
         [ 5.0396,  5.2072,  1.8779,  ..., -5.4419, -5.4425, -5.4426],
         [ 2.7506,  4.8974, -0.5766,  ..., -5.5188, -5.5188, -5.5189],
         ...,
         [ 4.2731,  5.7324, -0.9108,  ..., -5.5750, -5.5747, -5.5757],
         [ 4.7101,  3.7237, -0.0393,  ..., -5.8334, -5.8342, -5.8342],
         [ 3.5512,  4.9842, -1.4630,  ..., -7.0129, -7.0131, -7.0135]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=DynamicCache(), hidden_states=None, attentions=None)

### Test codes

In [116]:
outputs[0][0][0].tolist()

[9.02800178527832,
 6.477011203765869,
 3.8583221435546875,
 2.297102928161621,
 3.940321445465088,
 5.550507068634033,
 10.488454818725586,
 6.155864715576172,
 4.2740583419799805,
 5.1926093101501465,
 3.477968454360962,
 12.45777702331543,
 6.421036243438721,
 10.958100318908691,
 7.096643447875977,
 2.564802408218384,
 2.991946220397949,
 3.786231756210327,
 3.780236005783081,
 3.5180132389068604,
 2.7710275650024414,
 2.2379958629608154,
 3.1504194736480713,
 2.5894277095794678,
 2.004573106765747,
 10.963508605957031,
 5.233455181121826,
 3.2006664276123047,
 5.203616142272949,
 1.3057773113250732,
 7.537712574005127,
 3.2440617084503174,
 3.264991283416748,
 3.3815255165100098,
 3.7245144844055176,
 2.4964237213134766,
 2.717991828918457,
 4.410797119140625,
 4.603923320770264,
 4.56659460067749,
 2.7234909534454346,
 4.9857072830200195,
 3.8901281356811523,
 2.610147714614868,
 3.6849782466888428,
 3.4072296619415283,
 1.1063251495361328,
 4.706964015960693,
 2.8507752418518066

In [117]:
outputs[0][0][6]

tensor([ 3.5512,  4.9842, -1.4630,  ..., -7.0129, -7.0131, -7.0135],
       grad_fn=<SelectBackward0>)

# Scores

In [118]:
final_logits = model(input_ids).logits[0, -1]  # The last set of logits
final_logits.argmax()  # The position of the maximum

tensor(3729)

In [119]:
tokenizer.decode(final_logits.argmax())

' night'

In [120]:
top10_logits = torch.topk(final_logits, 10)
for index in top10_logits.indices:
    print(tokenizer.decode(index))

 night
 evening
 day
 morning
 winter
 afternoon
 Saturday
 Sunday
 Friday
 October


In [121]:
top10 = torch.topk(final_logits.softmax(dim=0), 10)
for value, index in zip(top10.values, top10.indices):
    print(f"{tokenizer.decode(index):<10} {value.item():.2%}")

 night     88.71%
 evening   4.30%
 day       2.19%
 morning   0.49%
 winter    0.45%
 afternoon 0.27%
 Saturday  0.25%
 Sunday    0.19%
 Friday    0.17%
 October   0.16%


In [122]:
output_ids = model.generate(input_ids, max_new_tokens=20)
decoded_text = tokenizer.decode(output_ids[0])

print("Input IDs", input_ids[0])
print("Output IDs", output_ids)
print(f"Generated text: {decoded_text}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Input IDs tensor([ 2132,   572,   264,  6319,   323, 13458,    88])
Output IDs tensor([[ 2132,   572,   264,  6319,   323, 13458,    88,  3729,    13,   576,
         12884,   572,  6319,   323,   279,  9956,   572,  1246,  2718,    13,
           576, 11174,   572, 50413,  1495,   323,   279]])
Generated text: It was a dark and stormy night. The sky was dark and the wind was howling. The rain was pouring down and the


In [123]:
beam_output = model.generate(
    input_ids,
    num_beams=5,
    max_new_tokens=30,
)

print(tokenizer.decode(beam_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


It was a dark and stormy night. The wind was howling, and the rain was pouring down. The sky was dark and gloomy, and the air was filled with the


## repetition_penalty

*   How much to penalize already generated tokens, avoiding repetition.
*   A good default value is 1.2.

## bad_words_ids

*  A list of tokens that should not be generated (e.g., to avoid generating offensive words).




In [124]:
beam_output = model.generate(
    input_ids,
    num_beams=5,
    repetition_penalty=2.0,
    max_new_tokens=38,
)

print(tokenizer.decode(beam_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


It was a dark and stormy night. The sky was filled with thunder and lightning, and the wind howled in the distance. It was raining cats and dogs, and the streets were covered in puddles of water.


#Sampling

In [125]:
# Setting the seed ensures we get the same results every time we run this code
set_seed(70)

sampling_output = model.generate(
    input_ids,
    do_sample=True,
    max_new_tokens=34,
    top_k=0,  # We'll come back to this parameter
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [126]:
tokenizer.decode(sampling_output[0])

'It was a dark and stormy night.Six pots of moisture laced the sky，pulling back fog and issuing a shared smell.\n16 km away they were cleaned and brought to her attention by'

In [127]:
sampling_output = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.4,
    max_new_tokens=40,
    top_k=0,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [128]:
tokenizer.decode(sampling_output[0])

'It was a dark and stormy night in the house, when we were all gathered around the fire. We were all looking forward to the night we were going to be together. I was sitting on the couch, watching the movie,'

In [129]:
sampling_output = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.001,
    max_new_tokens=40,
    top_k=0,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [130]:
tokenizer.decode(sampling_output[0])

'It was a dark and stormy night. The sky was dark and the wind was howling. The rain was pouring down and the lightning was flashing. The sky was dark and the wind was howling. The rain was pouring down'

In [131]:
sampling_output = model.generate(
    input_ids,
    do_sample=True,
    temperature=3.0,
    max_new_tokens=40,
    top_k=0,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [132]:
tokenizer.decode(sampling_output[0])

'It was a dark and stormy wheahkan exhilar swords seasHe Bd HibernateOthers Турية freed deploy Exhibition strtotimering finishing invadingmarker honoringЩ Uniform barracks Joan onde abbrev Mg/get铟 railway sticking Ant municipalities Kgforeach covering kin grown tacticalButtonText'

In [133]:
sampling_output = model.generate(
    input_ids,
    do_sample=True,
    max_new_tokens=40,
    top_k=5,
)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [134]:
tokenizer.decode(sampling_output[0])

'It was a dark and stormy night, and a young man named John had just arrived at the local hotel. His name was John Smith, and he was a traveler from a distant part of the world.\n\nAs John sat in his'

In [135]:
sampling_output = model.generate(
    input_ids,
    do_sample=True,
    max_new_tokens=40,
    top_p=0.94,
    top_k=0,
)

print(tokenizer.decode(sampling_output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


It was a dark and stormy night in America, home to a lone ship in the dark, the author spent her life of wandering and of survival in the woods that just happened to be adjoined by one of the deadliest and th


#Zero Shot

In [136]:
tokenizer.encode(" positive"), tokenizer.encode(" negative")

([6785], [8225])

In [139]:
def score(review):
    """Predict whether a review is positive or negative using perplexity
    
    This function compares the perplexity of two complete sentences
    to determine which is more likely.
    """
    prompt = f"""Question: Is the following review positive or negative about the movie?
    Review: {review} Answer:"""
    
    pos_text = prompt + " positive"
    neg_text = prompt + " negative"
    
    # Tokenize both complete texts
    pos_inputs = tokenizer(pos_text, return_tensors="pt")
    neg_inputs = tokenizer(neg_text, return_tensors="pt")
    
    # Calculate loss for each (lower loss = higher probability)
    with torch.no_grad():
        pos_loss = model(**pos_inputs, labels=pos_inputs["input_ids"]).loss.item()
        neg_loss = model(**neg_inputs, labels=neg_inputs["input_ids"]).loss.item()
    
    print(f"Positive loss: {pos_loss:.4f}, Negative loss: {neg_loss:.4f}")
    
    if pos_loss < neg_loss:
        print("Prediction: Positive")
    else:
        print("Prediction: Negative")
    
    return "positive" if pos_loss < neg_loss else "negative"

In [140]:
score("This movie was terrible!")

Positive loss: 3.9132, Negative loss: 3.7724
Prediction: Negative


'negative'

In [141]:
score("That movie was great!")

Positive loss: 4.0394, Negative loss: 4.0695
Prediction: Positive


'positive'

In [142]:
score("A complex yet wonderful film about the depravity of man")  # A mistake

Positive loss: 4.4471, Negative loss: 4.4178
Prediction: Negative


'negative'

In [107]:
prompt = """\
Translate English to Spanish:

English: I do not speak Spanish.
Spanish: No hablo español.

English: See you later!
Spanish: ¡Hasta luego!

English: Where is a good restaurant?
Spanish: ¿Dónde hay un buen restaurante?

English: What rooms do you have available?
Spanish: ¿Qué habitaciones tiene disponibles?

English: I like soccer
Spanish:"""
inputs = tokenizer(prompt, return_tensors="pt").input_ids
output = model.generate(
    inputs,
    max_new_tokens=30,
)

print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Translate English to Spanish:

English: I do not speak Spanish.
Spanish: No hablo español.

English: See you later!
Spanish: ¡Hasta luego!

English: Where is a good restaurant?
Spanish: ¿Dónde hay un buen restaurante?

English: What rooms do you have available?
Spanish: ¿Qué habitaciones tiene disponibles?

English: I like soccer
Spanish: Me gusta el fútbol

English: I'm sorry, I don't know.
Spanish: No, no, no, no, no
